In [2]:
import csv
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import math
import scipy.stats as stats

In [3]:
# 글씨 깨짐 방지
from matplotlib import rc  
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False 

In [4]:
# 빅쿼리에 접근

from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file("json파일경로")

In [5]:
from google.cloud import bigquery

client = bigquery.Client(
    credentials = credentials,
    project = credentials.project_id
)

In [1]:
# 점포 데이터 결측치
sql = """
SELECT
  *
FROM
  `restaurant_consumption.store`
WHERE
  (service_nm LIKE "%점"
    OR service_nm LIKE "커피%")
  AND service_nm NOT IN("편의점",
    "철물점",
    "%한복%",
    "%생활%")
"""
store = client.query(sql).to_dataframe()
store

In [2]:
store.isnull().sum()

In [4]:
# 유동인구 데이터 결측치
sql = """
SELECT
  *
FROM
  `restaurant_consumption.pop_by_road`
"""
population = client.query(sql).to_dataframe()
population

In [5]:
population.isnull().sum()

In [6]:
# 소득/소비 데이터 결측치
sql = """
SELECT
  *
FROM
  `restaurant_consumption.income_expense`
"""
income = client.query(sql).to_dataframe()
income

In [7]:
income.isnull().sum()

In [8]:
# 매출 데이터 결측치
sql = """
SELECT
  *
FROM
  `restaurant_consumption.sales`
WHERE
  (service_nm LIKE "%점" OR service_nm LIKE "커피%")
  AND service_nm NOT IN("편의점", "철물점", "%한복%", "%생활%")
"""
sales = client.query(sql).to_dataframe()
sales

In [9]:
sales.isnull().sum()

### 총 매출

In [10]:
sql = """
SELECT
  *
FROM
  `restaurant_consumption.sales`
WHERE
  year_quater_cd BETWEEN 20191 AND 20224
ORDER BY
  year_quater_cd
"""
sales = client.query(sql).to_dataframe()
sales

In [11]:
fig, ax = plt.subplots(ncols=2, nrows=1,  figsize=(10,5))

# 총 매출
sns.barplot(
    data = sales,
    x = "year_quater_cd",
    y = "month_sales_amount",
    ax = ax[0]
).set(
    title = "연도별 총 매출"
)
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=90)
# 총 매출 건수
plt.xticks(rotation=90)
sns.barplot(
    data = sales,
    x = "year_quater_cd",
    y = "month_sales_cnt",
    ax = ax[1]
).set(
    title = "연도별 총 매출 건수"
)
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=90)

In [132]:
sales = sales.drop(sales.iloc[:, 9:22], axis=1)

In [133]:
sales = sales.drop(["year_quater_cd"], axis=1)
sales = sales.drop(["autono_cd"], axis=1)
sales = sales.drop(["autono_nm"], axis=1)
sales = sales.drop(["service_cd"], axis=1)
sales = sales.drop(["service_nm"], axis=1)

In [12]:
sales

In [13]:
plt.figure(figsize=(15,12))
sns.heatmap(
    sales.corr(), 
    annot = True, 
    cmap = 'viridis',
    linewidth = 1
).set(
    title = "총 매출"
)

### 소득 소비

In [39]:
sql = """
SELECT
  *
FROM
  `restaurant_consumption.income_expense`
WHERE 
  year_quater_cd BETWEEN 20191 AND 20224
ORDER BY
  year_quater_cd
"""
incom = client.query(sql).to_dataframe()

In [14]:
income["total_expense"] = income["total_expense"]
income

In [15]:
plt.xticks(rotation=90)
sns.barplot(
    data = incom,
    x = "year_quater_cd",
    y = "month_avg_incom",
).set(
    title = "연도별 월평균 총 소득"
)

# >> 월 평균 총 소득이 일정하게 나옴 -> 데이터에 이상이 있다고 판단
# >> 소득/소비 데이터는 사용하지 않는 것이 좋을 듯 함

In [16]:
fig, ax = plt.subplots(ncols=2, nrows=1,  figsize=(10,5))

# 총 매출
sns.barplot(
    data = incom,
    x = "year_quater_cd",
    y = "month_avg_incom",
    ax = ax[0]
).set(
    title = "연도별 월평균 총 소득"
)
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=90)
# 총 매출 건수
# plt.xticks(rotation=90)
# sns.barplot(
#     data = sales,
#     x = "year_quater_cd",
#     y = "total_expense",
#     ax = ax[1]
# ).set(
#     title = "연도별 총 소비"
# )
# ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=90)




### 유동인구

In [17]:
sql = """
SELECT
  *
FROM
  `restaurant_consumption.pop_by_road`
WHERE
  year_quater_cd BETWEEN 20191 AND 20224
ORDER BY
  year_quater_cd
"""
population = client.query(sql).to_dataframe()
population

In [159]:
population = population.drop(population.iloc[:, 12:], axis=1)

In [160]:
population = population.drop(["year_quater_cd"], axis=1)
# population
population = population.drop(["autono_cd"], axis=1)
population = population.drop(["autono_nm"], axis=1)

In [18]:
population

In [19]:
population.corr()

In [20]:
plt.figure(figsize=(15,12))
sns.heatmap(
    population.corr(), 
    annot = True, 
    cmap = 'viridis',
    linewidth = 1
).set(
    title = "유동인구"
)

In [21]:
fig, ax = plt.subplots(ncols=2, nrows=1,  figsize=(10,5))

# 연도별 유동인구
sns.barplot(
    data = population,
    x = "year_quater_cd",
    y = "total_floating_pop",
    ax = ax[0]
).set(
    title = "연도별 총 유동인구"
)
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=90)
# 자치구별 유동인구
plt.xticks(rotation=90)
sns.barplot(
    data = population,
    x = "autono_nm",
    y = "total_floating_pop",
    ax = ax[1]
).set(
    title = "자치구별 총 유동인구"
)
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=90)


### 점포

In [22]:
sql = """
SELECT
  *
FROM
  `restaurant_consumption.store`
WHERE
  year_quater_cd BETWEEN 20191 AND 20224
ORDER BY
  year_quater_cd
"""
store = client.query(sql).to_dataframe()
store

In [23]:
fig, ax = plt.subplots(ncols=3, nrows=1,  figsize=(10,5))

# 총 점포
sns.barplot(
    data = store,
    x = "year_quater_cd",
    y = store['store_cnt'] + store['similar_store_cnt'],
    ax = ax[0]
).set(
    title = "전체 점포"
)
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=90)
# 개업 점포
plt.xticks(rotation=90)
sns.barplot(
    data = store,
    x = "year_quater_cd",
    y = "open_store_cnt",
    ax = ax[1]
).set(
    title = "개업 점포"
)
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=90)
# 폐업 점포
plt.xticks(rotation=90)
sns.barplot(
    data = store,
    x = "year_quater_cd",
    y = "close_store_cnt",
    ax = ax[2]
).set(
    title = "폐업 점포"
)
ax[2].set_xticklabels(ax[2].get_xticklabels(), rotation=90)

In [13]:
store = store.drop(["year_quater_cd"], axis=1)
store = store.drop(["autono_cd"], axis=1)
store = store.drop(["autono_nm"], axis=1)
store = store.drop(["service_cd"], axis=1)
store = store.drop(["service_nm"], axis=1)
store = store.drop(["open_rat"], axis=1)
store = store.drop(["close_rat"], axis=1)

In [24]:
store.corr()

In [25]:
sns.heatmap(
    store.corr(), 
    annot = True, 
    cmap = 'viridis',
    linewidth = 1
).set(
    title = "점포 수"
)

# 개/폐업

In [26]:
# 연도별 총 점포 수

sql = """
SELECT
  year_quater_cd,
  SUM(store_cnt+similar_store_cnt) AS store_cnt
FROM
  `restaurant_consumption.store`
WHERE
  (service_nm LIKE "%점" OR service_nm LIKE "커피%")
  AND service_nm NOT IN("편의점", "철물점", "%한복%", "%생활%")
  AND year_quater_cd BETWEEN 20191 AND 20224
GROUP BY
  year_quater_cd
ORDER BY
  1
"""
store_year = client.query(sql).to_dataframe()
store_year

In [27]:
plt.xticks(rotation=90)
sns.barplot(
    data = store_year,
    x = "year_quater_cd",
    y = "store_cnt",
).set(
    title = "연도별 음식업 총 점포 수"
)

In [28]:
# 연도별 개/폐업 점포 수
sql = """
SELECT
  year_quater_cd,
  SUM(open_store_cnt) AS open_store_cnt,
  SUM(close_store_cnt) AS close_store_cnt
FROM
  `restaurant_consumption.store`
WHERE
  year_quater_cd BETWEEN 20191 AND 20224
GROUP BY
  year_quater_cd
ORDER BY
  year_quater_cd
"""
store_oc = client.query(sql).to_dataframe()
store_oc["close/open"] = round((store_oc['close_store_cnt'] / store_oc['open_store_cnt']), 3)
store_oc

In [29]:
plt.xticks(rotation=90)
sns.barplot(
    data = store_oc,
    x = "year_quater_cd",
    y = "open_store_cnt",
).set(
    title = "연도별 개업 점포 수"
)

In [30]:
plt.xticks(rotation=90)
sns.barplot(
    data = store_oc,
    x = "year_quater_cd",
    y = "close_store_cnt",
).set(
    title = "연도별 폐업 점포 수"
)

In [31]:
fig, ax = plt.subplots(ncols=2, nrows=1,  figsize=(10,6))

# 개업
# plt.xticks(rotation=90)
sns.barplot(
    data = store_oc,
    x = "year_quater_cd",
    y = "open_store_cnt",
    ax = ax[0]
).set(
    title = "연도별 개업 점포 수"
)
ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=90)
# 폐업
plt.xticks(rotation=90)
sns.barplot(
    data = store_oc,
    x = "year_quater_cd",
    y = "close_store_cnt",
    ax = ax[1]
).set(
    title = "연도별 폐업 점포 수"
)
ax[1].set_xticklabels(ax[1].get_xticklabels(), rotation=90)

In [32]:
# 개업대비 폐업 비율
fig, ax = plt.subplots(ncols=1, nrows=1)
plt.xticks(rotation=90)
sns.barplot(
    data = store_oc,
    x = "year_quater_cd",
    y = "close/open",
).set(
    title = "연도별 폐업/개업 비율"
)
ax.axhline(y=1, color='red', linestyle='--', linewidth=2)


In [33]:
# 구별 총 점포 수

sql = """
SELECT
  autono_nm,
  SUM(store_cnt+similar_store_cnt) AS store_cnt
FROM
  `restaurant_consumption.store`
WHERE
  (service_nm LIKE "%점" OR service_nm LIKE "커피%")
  AND service_nm NOT IN("편의점", "철물점", "%한복%", "%생활%")
GROUP BY
  autono_nm
ORDER BY
  1
"""
store_gu = client.query(sql).to_dataframe()
store_gu

In [34]:
plt.figure(figsize=(10,5))
plt.xticks(rotation=90)
sns.barplot(
    data = store_gu,
    x = "autono_nm",
    y = "store_cnt",
).set(
    title = "구별 음식업 총 점포 수"
)

In [35]:
# 구별 개/폐업 점포 수

sql = """
SELECT
  autono_nm,
  SUM(open_store_cnt) AS open_store_cnt,
  SUM(close_store_cnt) AS close_store_cnt
FROM
  `restaurant_consumption.store`
WHERE
  (service_nm LIKE "%점"
    OR service_nm LIKE "커피%")
  AND service_nm NOT IN("편의점",
    "철물점",
    "%한복%",
    "%생활%")
GROUP BY
  autono_nm
ORDER BY
  1
"""
store_oc_gu = client.query(sql).to_dataframe()
store_oc_gu

In [36]:
plt.figure(figsize=(12, 6))
plt.xticks(rotation=90)

# 개업
sns.barplot(
    data = store_oc_gu,
    x = "autono_nm",
    y = "open_store_cnt",
)
# 폐업
sns.lineplot(
    data = store_oc_gu,
    x = "autono_nm",
    y = "close_store_cnt",
    color = 'blue',
    marker='o'
)
plt.legend(['open','close'])
# plt.legend(loc=2, bbox_to_anchor=(1, 1))

In [194]:
# 개/폐업에 영향을 미치는 요인 분석
# >> 가설 1. 유동인구가 음식업 개/폐업에 영향을 줄 것이다

In [47]:
# 소득소비
sql = """
SELECT
  autono_nm,
  SUM(month_avg_incom) AS month_avg_incom,
  SUM(total_expense) AS total_expense,
  SUM(food_total_expense) AS food_total_expense
FROM
  `restaurant_consumption.income_expense`
GROUP BY
  autono_nm
ORDER BY
  1
"""
income = client.query(sql).to_dataframe()
income

In [38]:
# 개/폐업
sql = """
SELECT
  autono_nm,
  SUM(store_cnt+similar_store_cnt) AS store_cnt,
  SUM(open_store_cnt) AS open_store_cnt,
  SUM(close_store_cnt) AS close_store_cnt
FROM
  `restaurant_consumption.store`
WHERE
  (service_nm LIKE "%점" OR service_nm LIKE "커피%")
  AND service_nm NOT IN("편의점", "철물점", "%한복%", "%생활%")
GROUP BY
  autono_nm
ORDER BY
  1
"""
store = client.query(sql).to_dataframe()
store

In [39]:
# 유동인구
sql = """
SELECT
  autono_nm,
  sum(total_floating_pop) as total_floating_pop,
  sum(male_floating_pop) as male_floating_pop,
  sum(female_floating_pop) as female_floating_pop,
  sum(gen10_floating_pop) as gen10_floating_pop,
  sum(gen20_floating_pop) as gen20_floating_pop,
  sum(gen30_floating_pop) as gen30_floating_pop,
  sum(gen40_floating_pop) as gen40_floating_pop,
  sum(gen50_floating_pop) as gen50_floating_pop,
  sum(gen60_floating_pop) as gen60_floating_pop
FROM
  `restaurant_consumption.pop_by_road`
group by
  autono_nm
order by
  autono_nm
"""
pop = client.query(sql).to_dataframe()
pop

In [199]:
x = store['open_store_cnt']
y = store['close_store_cnt']
stats.pearsonr(x,y)

# >> 개/폐업 사이에 강한 상관관계가 있음을 확인할 수 있다- (<- 완전 당연함)

PearsonRResult(statistic=0.9934813791918057, pvalue=3.3674389890511955e-23)

In [40]:
# 유동인구 ~ 개/폐업
pop_store = pd.concat([pop, store],axis=1, join='inner')
pop_store

In [41]:
pop_store = pop_store.drop(['autono_nm'], axis=1)
pop_store

In [42]:
pop_store.corr()

In [43]:
sns.heatmap(
    pop_store.corr(), 
    annot = True, 
    cmap = 'viridis',
    linewidth = 1
).set(
    title = "유동인구 ~ 개/폐업"
)

In [ ]:
# 음식점 개업(open_store_cnt)에 영향을 미치는 요인 (점포 수 제외)
# >> 30대, 40대 유동인구와 개업 점포 사이에 양의 상관관계가 있음을 확인할 수 있다

## 매출

In [48]:
sql = """
SELECT
  year_quater_cd,
  SUM(month_sales_amount) AS month_sales_amount
FROM
  `restaurant_consumption.sales`
WHERE
  (service_nm LIKE "%점" OR service_nm LIKE "커피%")
  AND service_nm NOT IN("편의점", "철물점", "%한복%", "%생활%")
GROUP BY
  year_quater_cd
ORDER BY
  year_quater_cd
"""
sales = client.query(sql).to_dataframe()
sales

In [49]:
sql = """
SELECT
  year_quater_cd,
  SUM(month_sales_amount) AS month_sales_amount
FROM
  `restaurant_consumption.sales`
WHERE
  (service_nm LIKE "%점" OR service_nm LIKE "커피%")
  AND service_nm NOT IN("편의점", "철물점", "%한복%", "%생활%")
  AND year_quater_cd BETWEEN 20221 AND 20224
GROUP BY
  year_quater_cd
ORDER BY
  year_quater_cd
"""
sales_22 = client.query(sql).to_dataframe()
sales_22

In [50]:
plt.bar(sales['year_quater_cd'], sales['month_sales_amount'])
plt.xlabel("year")
plt.ylabel("sales")
plt.title("총 매출")

In [51]:
plt.xticks(rotation=90)
plt.title("Total Sales")
sns.barplot(
    data = sales,
    x = "year_quater_cd",
    y = "month_sales_amount"
)

In [52]:
# 연도, 분기별 총 매출

plt.figure(figsize=(7.5,6))
plt.subplot(221)
sns.barplot(
    data = sales_19,
    x = "year_quater_cd",
    y = "month_sales_amount"
)
plt.subplot(222)
sns.barplot(
    data = sales_20,
    x = "year_quater_cd",
    y = "month_sales_amount"
)
plt.subplot(223)
sns.barplot(
    data = sales_21,
    x = "year_quater_cd",
    y = "month_sales_amount"
)
plt.subplot(224)
sns.barplot(
    data = sales_22,
    x = "year_quater_cd",
    y = "month_sales_amount"
)
axes.title("2019~2022 총 매출")

In [174]:
# >> 1분기보다 4분기에 매출이 확 오르는걸 볼 수 있음
# 왜?.?
# 가설 1. 구별 매출 변화가 총 매출에 영향을 미쳤을 것이다.
# 가설 2. 업종별 매출 변화가 총 매출에 영향을 미쳤을 것이다.

### 가설 1. 구별 매출 변화가 총 매출에 영향을 미쳤을 것이다.

In [53]:
sql = """
SELECT
  autono_nm,
  SUM(month_sales_amount) AS month_sales_amount
FROM
  `restaurant_consumption.sales`
WHERE
  (service_nm LIKE "%점" OR service_nm LIKE "커피%")
  AND service_nm NOT IN("편의점", "철물점", "%한복%", "%생활%")
  AND year_quater_cd BETWEEN 20221 and 20224
GROUP BY
  autono_nm
ORDER BY
  autono_nm
"""
sales_gu_22 = client.query(sql).to_dataframe()
sales_gu_22

In [69]:
# plt.figure(figsize=(10,8))
plt.xticks(rotation=90)

plt.plot(sales_gu_19['autono_nm'], sales_gu_19['month_sales_amount'], label="sales_19")
plt.plot(sales_gu_19['autono_nm'], sales_gu_20['month_sales_amount'], label="sales_20")
plt.plot(sales_gu_19['autono_nm'], sales_gu_21['month_sales_amount'], label="sales_21")
plt.plot(sales_gu_19['autono_nm'], sales_gu_22['month_sales_amount'], label="sales_22")
plt.legend(loc=2, bbox_to_anchor=(1, 1))
# plt.xlabel('자치구')
# plt.ylabel('총 매출')
plt.title('자치구 매출 ~ 총 매출')
plt.show()

In [ ]:
# >> 1, 4분기 총 매출 변화가 자치구별 총 매출에 영향을 받지 않는 것으로 보임
# >> 가설 1 기각

### 가설 2. 업종별 매출 변화가 총 매출에 영향을 미쳤을 것이다.

In [68]:
sql = """
SELECT
  service_nm,
  SUM(month_sales_amount) AS month_sales_amount
  
FROM
  `restaurant_consumption.sales`
WHERE
  (service_nm LIKE "%점" OR service_nm LIKE "커피%")
  AND service_nm NOT IN("편의점", "철물점", "%한복%", "%생활%")
  AND year_quater_cd BETWEEN 20221 and 20224
GROUP BY
  service_nm
ORDER BY
  service_nm
"""
sales_store_22 = client.query(sql).to_dataframe()
sales_store_22

In [67]:
plt.xticks(rotation=90)
plt.plot(sales_store_19['service_nm'], sales_store_19['month_sales_amount'], label="sales_19")
plt.plot(sales_store_20['service_nm'], sales_store_20['month_sales_amount'], label="sales_20")
plt.plot(sales_store_21['service_nm'], sales_store_21['month_sales_amount'], label="sales_21")
plt.plot(sales_store_22['service_nm'], sales_store_22['month_sales_amount'], label="sales_22")

plt.legend(loc=2, bbox_to_anchor=(1, 1))

In [ ]:
# >> 1, 4분기 총 매출 변화가 업종별 총 매출에 영향을 받는 것으로 보임
# >> 가설 2 채택

In [ ]:
# >> 그렇다면, 업종별 매출에 영향을 끼치는 요인을 알아보면 

## 자치구별 음식업종 비율

In [66]:
# 구별 전체 음식점 수
sql = """
SELECT
  autono_nm,
  SUM(store_cnt+similar_store_cnt) as total_cnt
FROM
  `restaurant_consumption.store`
WHERE
  (service_nm LIKE "%점" OR service_nm LIKE "커피%")
  AND service_nm NOT IN("편의점", "철물점", "한복점", "재생용품 판매점")
GROUP BY
  autono_nm
ORDER BY
  autono_nm
"""
store_all = client.query(sql).to_dataframe()
store_all

In [65]:
store_total_cnt = list(store_all["total_cnt"])
store_total_cnt

In [64]:
# 강남구 음식업별 점포 수
sql = """
SELECT
  service_nm,
  SUM(store_cnt+similar_store_cnt) as total_cnt
FROM
  `restaurant_consumption.store`
WHERE
  (service_nm LIKE "%점" OR service_nm LIKE "커피%")
  AND service_nm NOT IN("편의점", "철물점", "한복점", "재생용품 판매점")
  and autono_nm like '%강남%'
GROUP BY
  service_nm
ORDER BY
  service_nm
"""
store_gangnam = client.query(sql).to_dataframe()

# 업종 점포 수 / (자치구별)전체 업종 수
store_gangnam["total_rat"] = round(store_gangnam["total_cnt"] / store_total_cnt[0], 3)
store_gangnam

## 가설 설정

### 가설1. 유동인구와 개업점포 사이에 상관관계가 있다

In [63]:
sql = """
SELECT
  year_quater_cd,
  SUM(open_store_cnt) AS open_store_cnt
FROM
  `restaurant_consumption.store`
WHERE
  year_quater_cd BETWEEN 20191 AND 20224
GROUP BY
  year_quater_cd
ORDER BY
  year_quater_cd
"""
store_open = client.query(sql).to_dataframe()
store_open

In [62]:
sql = """
SELECT
  year_quater_cd,
  SUM(total_floating_pop) AS total_floating_pop
FROM
  `restaurant_consumption.pop_by_road`
WHERE
  year_quater_cd BETWEEN 20191 AND 20224
GROUP BY
  year_quater_cd
ORDER BY
  year_quater_cd
"""
population = client.query(sql).to_dataframe()
population

In [61]:
x = store_open["open_store_cnt"]
y = population["total_floating_pop"]
stats.pearsonr(x, y)

# >> 유의확률이 0.676으로 유의수준 0.05보다 크기 때문에 대립가설을 기각하고 귀무가설(유동인구와 개업 점포 수 사이에 관계가 없다)을 채택한다
# >> 총 유동인구와 개업 점포 사이에는 상관관계가 없는 것을 확인할 수 있다

### 가설2. 유동인구와 매출 사이에 상관관계가 있다

In [60]:
population

In [59]:
sql = """
SELECT
  year_quater_cd,
  SUM(month_sales_amount) AS month_sales_amount
FROM
  `restaurant_consumption.sales`
WHERE
  year_quater_cd BETWEEN 20191 AND 20224
GROUP BY
  year_quater_cd
ORDER BY
  year_quater_cd
"""
sales = client.query(sql).to_dataframe()
sales

In [58]:
x = sales["month_sales_amount"]
y = population["total_floating_pop"]
stats.pearsonr(x, y)

# >> 유의확률이 0.842로 유의수준 0.05보다 크기 때문에 대립가설을 기각하고 귀무가설(전체 유동인구와 총 매출 사이에 관계가 없다)을 채택한다
# >> 총 유동인구와 개업 점포 사이에는 상관관계가 없는 것을 확인할 수 있다

### 가설3. 특정 연령대 유동인구와 매출 사이에 관계가 있다

In [57]:
sql = """
SELECT
  year_quater_cd,
  SUM(total_floating_pop) AS total_floating_pop,
  SUM(gen10_floating_pop) AS gen10_floating_pop,
  SUM(gen20_floating_pop) AS gen20_floating_pop,
  SUM(gen30_floating_pop) AS gen30_floating_pop,
  SUM(gen40_floating_pop) AS gen40_floating_pop,
  SUM(gen50_floating_pop) AS gen50_floating_pop,
  SUM(gen60_floating_pop) AS gen60_floating_pop,
FROM
  `restaurant_consumption.pop_by_road`
WHERE
  year_quater_cd BETWEEN 20191 AND 20224
GROUP BY
  year_quater_cd
ORDER BY
  year_quater_cd
"""
pop_age = client.query(sql).to_dataframe()
pop_age

In [56]:
sales_popAge = pd.concat([sales,pop_age],axis=1, join='inner')
sales_popAge

In [103]:
sales_popAge = sales_popAge.drop(["year_quater_cd"], axis=1)

In [55]:
sales_popAge

In [54]:
plt.figure(figsize=(15,12))
sns.heatmap(
    sales_popAge.corr(), 
    annot = True, 
    cmap = 'viridis',
    linewidth = 1
).set(
    title = "연령대별 유동인구 ~ 총 매출"
)

# >> 총 매출의 상관계수가 0.7 이상인 연령대 없음
# >> 특정 연령대와 총 매출 사이에는 관계가 없음을 알 수 있다